In [1]:
from pathlib import Path
import pandas as pd

data_path = Path('c:\\Users\\mcvitano01\\OneDrive - JPS Health Network\\registries\\hiv-art-resistance\\data')
monogram_path = data_path.joinpath('monogram')
hivdb_path = data_path.joinpath('stanford_hivdb')

# This is a switch
#   --> Parsing PDFs is a lengthy, compute-intensive process
#   --> In contrast, downstream functions that take {df} as input complete in < 60 seconds
try:
    df = pd.read_csv(monogram_path.joinpath('genotypic-tests-2021-12-31.tsv'), 
                     dtype={'PAT_MRN_ID': str}, sep='\t')

    rerun_pdf_parsing = False
except:
    rerun_pdf_parsing = True
    pass

## Step 1
### Manually download test result PDFs from Monogram site

## Step 2(a)
### Parse PDFs into dataframes
* *\> 15 minutes to run*

In [2]:
from src.utils import parse_genotypic_reports

if rerun_pdf_parsing:
    # Geneseq
    geneseq = parse_genotypic_reports(
        monogram_path.joinpath('geneseq'), test_type='geneseq')

    geneseq.to_csv(monogram_path.joinpath(
        'geneseq/geneseq-2021-12-31.tsv'), index=False, sep='\t')

    # Genosure-MG
    genosure_mg = parse_genotypic_reports(
        monogram_path.joinpath('genosure-mg'), test_type='genosure-mg')

    genosure_mg.to_csv(monogram_path.joinpath(
        'genosure-mg/genosure-mg-2021-12-31.tsv'), index=False, sep='\t')

    # Genosure-PRIme
    genosure_prime = parse_genotypic_reports(
        monogram_path.joinpath('genosure-prime'), test_type='genosure-prime')

    genosure_prime.to_csv(monogram_path.joinpath(
        'genosure-prime/genosure-prime-2021-12-31.tsv'), index=False, sep='\t')

    # Genosure-Archive
    genosure_archive = parse_genotypic_reports(
        monogram_path.joinpath('genosure-archive'), test_type='genosure-archive')

    genosure_archive.to_csv(monogram_path.joinpath(
        'genosure-archive/genosure-archive-2021-12-31.tsv'), index=False, sep='\t')

    # Phenosense-GT
    phenosense_gt = parse_genotypic_reports(
        monogram_path.joinpath('phenosense-gt'), test_type='phenosense-gt')

    phenosense_gt.to_csv(monogram_path.joinpath(
        'phenosense-gt/phenosense-gt-2021-12-31.tsv'), index=False, sep='\t')


    # Combine
    df = pd.concat([geneseq, genosure_mg, genosure_prime, genosure_archive, phenosense_gt])
    df.sort_values(['PAT_MRN_ID', 'COLLECTED_DATE'], inplace=True)
    df.reset_index(inplace=True, drop=True)

    # Write to disk
    df.to_csv(monogram_path.joinpath(
        'genotypic-tests-2021-12-31.tsv'), index=False, sep='\t')

## Step 2(b)
### Chart review for missing MRNs

In [3]:
from src.utils import add_chart_reviewed_mrns

if rerun_pdf_parsing:
    df = add_chart_reviewed_mrns(df, monogram_path.joinpath('completed-mrn-for-chart-review.csv'))

    # Write to disk
    df.to_csv(monogram_path.joinpath(
        'genotypic-tests-2021-12-31.tsv'), index=False, sep='\t')

## Step 3
### Transform lists of mutations into loci-specific position dataframes

In [5]:
from src.utils import melt_loci_specific_mutations_to_position

# Transform lists of mutations at each locus to a loci-specific position dataframe
#   with one row per position (in the HIV-1 DNA for that locus)
#   rather than one row per test accession (each with a list of mutations)
rt_position = melt_loci_specific_mutations_to_position(df, loci='RT')
pr_position = melt_loci_specific_mutations_to_position(df, loci='PR')
insti_position = melt_loci_specific_mutations_to_position(df, loci='INSTI')

Extracting positions of mutations within the RT_LIST locus.
All values of "None" must be set to missing to allow 
     filling mutations forward over time (within patient).

Dropped
  - 1404 without observed mutations at the RT locus
  - 0 incomplete test accessions
  - 2 test accessions with no MRN

Extracting positions of mutations within the PR_LIST locus.
All values of "None" must be set to missing to allow 
     filling mutations forward over time (within patient).

Dropped
  - 3714 without observed mutations at the PR locus
  - 0 incomplete test accessions
  - 1 test accessions with no MRN

Extracting positions of mutations within the INSTI_LIST locus.
All values of "None" must be set to missing to allow 
     filling mutations forward over time (within patient).

Dropped
  - 5259 without observed mutations at the INSTI locus
  - 0 incomplete test accessions
  - 0 test accessions with no MRN



## Step 4
### Add Stanford HIVdb comments to each position

In [6]:
from src.utils import load_hivdb_comments, replicate_comments_over_amino_acids

# Add Stanford HIVDB mutation comments to RT position dataframe
rt_comments = load_hivdb_comments(hivdb_path, loci='RT')
rt_comments = replicate_comments_over_amino_acids(rt_comments)

rt_position = rt_position.merge(rt_comments,
                                 left_on=['POSITION', 'AA'], 
                                 right_on=['POSITION', 'AA'], how='left')

# Add Stanford HIVDB mutation comments to PR/PI position dataframe
pr_comments = load_hivdb_comments(hivdb_path, loci='PI')
pr_comments = replicate_comments_over_amino_acids(pr_comments)

pr_position = pr_position.merge(pr_comments,
                                 left_on=['POSITION', 'AA'], 
                                 right_on=['POSITION', 'AA'], how='left')

# Add Stanford HIVDB mutation comments to INSTI position dataframe
insti_comments = load_hivdb_comments(hivdb_path, loci='INSTI')
insti_comments = replicate_comments_over_amino_acids(insti_comments)

insti_position = insti_position.merge(insti_comments,
                                 left_on=['POSITION', 'AA'], 
                                 right_on=['POSITION', 'AA'], how='left')

## Step 5
### Score mutations at each position according to Standord HIVdb's "simple" rules

In [7]:
from src.utils import load_hivdb_scores, score_positioned_mutations

# Load Stanford HIVDB mutation scores for each locus
rt_scores = load_hivdb_scores(hivdb_path, loci='RT', rule_type='simple')
# rename Tenofovir disoproxil in Stanford (TDF) to match Monogram (TFV)
rt_scores = [col.replace('TDF', 'TFV') for col in rt_scores.columns]

pr_scores = load_hivdb_scores(hivdb_path, loci='PR', rule_type='simple')
insti_scores = load_hivdb_scores(hivdb_path, loci='INSTI', rule_type='simple')

# Score each position
rt_scored = score_positioned_mutations(rt_position, rt_scores)
pr_scored = score_positioned_mutations(pr_position, pr_scores)
insti_scored = score_positioned_mutations(insti_position, insti_scores)

# Write to disk
rt_scored.to_csv(data_path.joinpath('rt-position-scored.tsv'), sep='\t', index=False)
pr_scored.to_csv(data_path.joinpath('pr-position-scored.tsv'), sep='\t', index=False)
insti_scored.to_csv(data_path.joinpath('insti-position-scored.tsv'), sep='\t', index=False)

## Step 6
### Rescore complex interactions

The estimated level of resistance to a drug is determined by adding up the penalty scores associated with each of the DRMs present in a submitted sequence.  

Once the total score is calculated the estimated level of resistance can be calculated as follows:
* Susceptible: Total score 0 to 9
* Potential low-level resistance: Total score 10 to 14
* Low-level resistance: Total score 15 to 29
* Intermediate resistance: Total score 30 to 59
* High-level resistance: Total score >= 60

Interpretation  
* "Susceptible" indicates no evidence of reduced ARV susceptibility compared with a wild-type virus.
* "Potential low-level resistance" indicates that the sequence may contain mutations indicating previous ARV exposure or may contain mutation that are associated with drug resistance only when they occur with additional mutations.
* "Low-level resistance" indicates that there that the virus encoded by the submitted sequence may have reduced in vitro ARV susceptibility or that patients harboring viruses with the submitted mutations may have a suboptimal virological response to treatment with the ARV.
* "Intermediate resistance" indicates a high likelihood that a drug's activity will be reduced but that the drug will likely retain significant remaining antiviral activity.
* "High-level resistance" indicates that the predicted level of resistance is similar to those observed in viruses with the highest levels of in vitro drug resistance or that clinical data exist demonstrating that patients infected with viruses having such mutations usually have little or no virological response to treatment with the ARV.

Complex Rules  
* Some combinations of DRMs receive penalty scores that are **added to the total score** for a drug. For example:
    + The RT mutations L74I/V (L74I or L74V) and M184I/V (M184I or M184V) have penalty scores of 30 and 15, respectively for abacavir (ABC). In addition, L74I/V + M184/V has a penalty score of 15 for ABC. Therefore, a sequence with L74V + M184V will have a total penalty score of 60 (30 + 15 + 15) which is translated into high-level ABC resistance.
    + The PR mutations M46I/L, I54V, and V82A have penalty scores of 10, 15, and 30 respectively for lopinavir/r (LPV/r). M54A/L/M/S/T/V + V82A/C/F/M/L/S/T/V has a penalty score of 10 for LPV/r. Therefore, a sequence with M46IL + I54V + V82A will have a total penalty score of 65 (10 + 15 + 30 + 10) which is translated into high-level LPV/r resistance.  

Multiple Mutations at the Same Position
* When there is a mixture of two mutations at the same position, the mutation associated with the **largest penalty is scored**. Therefore, if a mutation associated with a negative penalty score is present in a mixture with the wildtype amino acid at that position, there will be no negative penalty score.

In [9]:
from src.utils import load_hivdb_scores, calculate_total_resistance

rt_interactions = load_hivdb_scores(hivdb_path, loci='RT', rule_type='Complex')
# rename Tenofovir disoproxil in Stanford (TDF) to match Monogram (TFV)
rt_interactions = [col.replace('TDF', 'TFV') for col in rt_interactions.columns]
rt_rescored = calculate_total_resistance(rt_scored, rt_interactions)

pr_interactions = load_hivdb_scores(hivdb_path, loci='PR', rule_type='Complex')
pr_rescored = calculate_total_resistance(pr_scored, pr_interactions)

insti_interactions = load_hivdb_scores(hivdb_path, loci='INSTI', rule_type='Complex')
insti_rescored = calculate_total_resistance(insti_scored, insti_interactions)

# Combine
total_resistance = pd.concat([rt_rescored,
                              pr_rescored,
                              insti_rescored])

total_resistance.sort_values(['PAT_MRN_ID', 'COLLECTED_DATE'], inplace=True)
total_resistance.reset_index(inplace=True, drop=True)

# Write to disk
total_resistance.to_csv(
    data_path.joinpath('total-resistance-2021-12-31.tsv'), sep='\t', index=False)

In [15]:
total_resistance.groupby(['ARV', 'RESISTANCE_CATEGORY'])['PAT_MRN_ID'].nunique().head(50)

ARV    RESISTANCE_CATEGORY
3TC    High                    436
       Intermediate             11
       Low                       5
       Potential low-level      38
       Suceptible             2511
ABC    High                    108
       Intermediate             88
       Low                     381
       Potential low-level      19
       Suceptible             2450
ATV/R  High                     28
       Intermediate             17
       Low                      38
       Potential low-level      41
       Suceptible             1672
AZT    High                    111
       Intermediate             88
       Low                      59
       Potential low-level     126
       Suceptible             2365
BIC    High                      3
       Intermediate              8
       Low                       3
       Potential low-level      19
       Suceptible              697
CAB    High                      9
       Intermediate              4
       Low                  